In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")
del stats["Unnamed: 0"]
stats_all_players = stats.fillna(0)
#mvp_players = stats_all_players[(stats_all_players["FG%"] > 0.384) & (stats_all_players["MP"] > 25.6) & (stats_all_players["FT%"] > 0.422) & (stats_all_players["PTS"] > 4.7) & (stats_all_players["VORP"] > 1.1)& (stats_all_players["W/L%"] > 0.22)]
mvp_players = stats_all_players

In [3]:
stats_2024 = pd.read_csv("2024_stats.csv")
del stats_2024["Unnamed: 0"]

In [4]:
stats_2024 = stats_2024.fillna(0)
#stats_2024 = stats_2024[(stats_2024["FG%"] > 0.384) & (stats_2024["MP"] > 25.6) & (stats_2024["FT%"] > 0.422) & (stats_2024["PTS"] > 4.7) & (stats_2024["VORP"] > 1.1)& (stats_2024["W/L%"] > 0.22)]
condition = (stats_2024['W'] + stats_2024['L']) - stats_2024['G'] < 17
stats_2024 = stats_2024[condition]

In [5]:
# pd.set_option('display.max_rows', None)
# len(stats_2024["Player"])

In [6]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS','PER','TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']

#Predicting on 2024 players stats. Training model on data from previous years (1990 - 2023)
train = mvp_players
test = stats_2024

X_train = train[predictors]
y_train = train["Share"]

X_test = test[predictors]
# y_test = test["Share"]

model = XGBRegressor(learning_rate = 0.05, max_depth = 7, subsample = 0.5)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_XGB = pd.concat([test, predicted_winners], axis = 1)
combinations_XGB = combinations_XGB.sort_values("Predictions", ascending = False)
# combinations = combinations.sort_values("Share", ascending = False)
# combinations["Rank"] = list(range(1, combinations.shape[0] + 1))
# combinations = combinations.sort_values("Predictions", ascending = False)
# combinations["Predicted_Rank"] = list(range(1, combinations.shape[0] + 1))

In [7]:
combinations_XGB.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
67,Nikola Jokić,C,28,DEN,74,74,34.6,10.4,18.0,0.579,...,10.1,Denver Nuggets,53,24,0.688,0.5,114.3,109.6,4.78,0.981386
123,Shai Gilgeous-Alexander,PG,25,OKC,71,71,34.4,10.7,19.9,0.539,...,6.8,Oklahoma City Thunder,52,24,0.684,1.0,119.9,113.3,6.72,0.732287
7,Giannis Antetokounmpo,PF,29,MIL,71,71,35.2,11.5,18.8,0.613,...,7.0,Milwaukee Bucks,47,29,0.618,0.0,120.0,116.8,2.79,0.577098
30,Luka Dončić,PG,24,DAL,67,67,37.4,11.5,23.6,0.486,...,7.6,Dallas Mavericks,46,30,0.605,0.0,118.3,115.8,2.66,0.463605
243,Domantas Sabonis,C,27,SAC,76,76,35.8,7.8,13.1,0.599,...,5.8,Sacramento Kings,44,32,0.579,4.0,117.0,115.8,1.64,0.118853
166,Jayson Tatum,PF,25,BOS,71,71,35.8,9.2,19.4,0.474,...,4.6,Boston Celtics,60,16,0.789,0.0,121.0,109.3,11.33,0.103445
185,Jalen Brunson,PG,27,NYK,71,71,35.1,10.0,20.9,0.477,...,4.7,New York Knicks,45,31,0.592,15.0,112.5,107.9,4.27,0.062687
334,Anthony Edwards,SG,22,MIN,73,72,35.2,9.2,19.9,0.461,...,3.5,Minnesota Timberwolves,53,23,0.697,0.0,113.1,106.0,7.05,0.045992
105,Tyrese Haliburton,PG,23,IND,64,63,32.2,7.4,15.5,0.476,...,4.8,Indiana Pacers,43,34,0.558,4.5,122.7,120.3,2.13,0.029001
219,Anthony Davis,C,30,LAL,72,72,35.8,9.4,17.0,0.554,...,4.5,Los Angeles Lakers,44,33,0.571,4.5,117.9,117.5,0.80,0.027213


In [8]:
combinations_XGB.head(5).to_csv("XGB_predictions.csv")

In [9]:
model = RandomForestRegressor(bootstrap=True, max_depth=40, max_features = 'sqrt', min_samples_leaf=2, min_samples_split=2, n_estimators = 200)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_RF = pd.concat([test, predicted_winners], axis = 1)
combinations_RF = combinations_RF.sort_values("Predictions", ascending = False)

In [10]:
combinations_RF.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
67,Nikola Jokić,C,28,DEN,74,74,34.6,10.4,18.0,0.579,...,10.1,Denver Nuggets,53,24,0.688,0.5,114.3,109.6,4.78,0.814201
123,Shai Gilgeous-Alexander,PG,25,OKC,71,71,34.4,10.7,19.9,0.539,...,6.8,Oklahoma City Thunder,52,24,0.684,1.0,119.9,113.3,6.72,0.530275
7,Giannis Antetokounmpo,PF,29,MIL,71,71,35.2,11.5,18.8,0.613,...,7.0,Milwaukee Bucks,47,29,0.618,0.0,120.0,116.8,2.79,0.501736
30,Luka Dončić,PG,24,DAL,67,67,37.4,11.5,23.6,0.486,...,7.6,Dallas Mavericks,46,30,0.605,0.0,118.3,115.8,2.66,0.377862
166,Jayson Tatum,PF,25,BOS,71,71,35.8,9.2,19.4,0.474,...,4.6,Boston Celtics,60,16,0.789,0.0,121.0,109.3,11.33,0.157584
219,Anthony Davis,C,30,LAL,72,72,35.8,9.4,17.0,0.554,...,4.5,Los Angeles Lakers,44,33,0.571,4.5,117.9,117.5,0.80,0.070373
243,Domantas Sabonis,C,27,SAC,76,76,35.8,7.8,13.1,0.599,...,5.8,Sacramento Kings,44,32,0.579,4.0,117.0,115.8,1.64,0.060665
185,Jalen Brunson,PG,27,NYK,71,71,35.1,10.0,20.9,0.477,...,4.7,New York Knicks,45,31,0.592,15.0,112.5,107.9,4.27,0.055038
334,Anthony Edwards,SG,22,MIN,73,72,35.2,9.2,19.9,0.461,...,3.5,Minnesota Timberwolves,53,23,0.697,0.0,113.1,106.0,7.05,0.053646
437,Kevin Durant,PF,35,PHO,69,69,37.1,10.1,19.2,0.528,...,4.2,Phoenix Suns,45,31,0.592,3.0,116.9,113.9,2.61,0.043413


In [11]:
combinations_RF.head(5).to_csv("RF_predictions.csv")